In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [11]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer

In [12]:
# datasets = load_dataset("conll2003")

In [13]:
datasets = load_dataset('dataset_loader.py', 'sample')

0 examples [00:00, ? examples/s]data_dir:  {'train': '/Users/nicholasmartin/.cache/huggingface/datasets/downloads/38c090853a9af4b7f11dc8b1369f03f0cdbb7941a89916a40968bfdd0de0a8e4', 'validation': '/Users/nicholasmartin/.cache/huggingface/datasets/downloads/66aa9224ada692c0b70f87f8a5b28261123d5759f6e5e504dbc08be8748c0c2d', 'test': '/Users/nicholasmartin/.cache/huggingface/datasets/downloads/6294e07d5a68b2f68aada58f113cb5bc726eeced462b15f9ee7caf994dec0740'}
0 examples [00:00, ? examples/s]Dataset dutch_neutrality_corpus_dataset downloaded and prepared to /Users/nicholasmartin/.cache/huggingface/datasets/dutch_neutrality_corpus_dataset/sample/1.0.0/16587180dddf2cab76bf5b579914f85f38a749ef6d34c71a87a06a418b8d90dd. Subsequent calls will reuse this data.


In [14]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'tokens', 'class_labels'],
        num_rows: 1212
    })
    validation: Dataset({
        features: ['text', 'tokens', 'class_labels'],
        num_rows: 260
    })
    test: Dataset({
        features: ['text', 'tokens', 'class_labels'],
        num_rows: 260
    })
})

In [59]:
# datasets["train"][0]

In [62]:
# datasets["train"].features[f"ner_tags"]
label_list = datasets["train"].features[f"class_labels"].feature.names

In [63]:
label_list

['SUBJ', 'NEUT']

In [64]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [66]:
tokenizer(
    datasets["train"][0]['tokens'],
    is_split_into_words=True
)

{'input_ids': [101, 7632, 3501, 2003, 8254, 5104, 2807, 25312, 6593, 2666, 6767, 2953, 5831, 12079, 3158, 2139, 3769, 15859, 29472, 2818, 6971, 2063, 6102, 13719, 2112, 28418, 11867, 1999, 2139, 26922, 2063, 27829, 2121, 4372, 8254, 5104, 2997, 2455, 20806, 15992, 2112, 28418, 6767, 2953, 5831, 12079, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [67]:
example = datasets["train"][4]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'de', 'tr', '##adi', '##ties', 'en', 'de', 'japan', '##se', 'same', '##nl', '##eving', 'vera', '##nder', '##en', 'ni', '##et', 's', '##nel', '[SEP]']


In [68]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8, None]


In [69]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["class_labels"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

20 20


In [70]:
label_all_tokens = True

In [71]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["class_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [79]:
# tokenized_datasets['train'][1]

In [73]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

100%|██████████| 2/2 [00:00<00:00,  3.77ba/s]


In [76]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [77]:
args = TrainingArguments(
    "experiments",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

In [80]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [81]:
metric = load_metric("seqeval")

In [82]:
labels = [label_list[i] for i in example["class_labels"]]
metric.compute(predictions=[labels], references=[labels])

{'EUT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [83]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [84]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["train"],  # Change to 'validation'...
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [85]:
trainer.train()

100%|██████████| 109/109 [14:30<00:00,  4.66s/it]


  0%|          | 0/109 [00:00<?, ?it/s]


  2%|▏         | 2/109 [00:01<01:16,  1.39it/s]


  3%|▎         | 3/109 [00:05<02:47,  1.58s/it]


  4%|▎         | 4/109 [00:07<03:22,  1.93s/it]


  5%|▍         | 5/109 [00:08<02:53,  1.67s/it]


  6%|▌         | 6/109 [00:09<02:33,  1.49s/it]


  6%|▋         | 7/109 [00:11<02:32,  1.49s/it]


  7%|▋         | 8/109 [00:13<02:58,  1.76s/it]


  8%|▊         | 9/109 [00:17<03:43,  2.24s/it]


  9%|▉         | 10/109 [00:18<03:25,  2.07s/it]


 10%|█         | 11/109 [00:20<03:15,  2.00s/it]


 11%|█         | 12/109 [00:22<03:09,  1.96s/it]


 12%|█▏        | 13/109 [00:23<02:38,  1.65s/it]


 13%|█▎        | 14/109 [00:26<03:13,  2.04s/it]


 14%|█▍        | 15/109 [00:27<02:49,  1.80s/it]


 15%|█▍        | 16/109 [00:28<02:28,  1.60s/it]


 16%|█▌        | 17/109 [00:31<03:06,  2.02s/it]


 17%|█▋        | 18/109 [00:33<02:52,  1.89s/it]


 17%|█▋        | 19/109 [00:35<03:01,  2.01s/it

TrainOutput(global_step=109, training_loss=0.49466771816988603, metrics={'train_runtime': 1061.3375, 'train_samples_per_second': 0.103, 'epoch': 1.0})

In [86]:
trainer.evaluate()

100%|██████████| 109/109 [02:52<00:00,  1.58s/it]


{'eval_loss': 0.48548054695129395,
 'eval_precision': 0.49191685912240185,
 'eval_recall': 0.05343367826904986,
 'eval_f1': 0.09639644736097754,
 'eval_accuracy': 0.8085934754803435,
 'eval_runtime': 173.2819,
 'eval_samples_per_second': 9.995,
 'epoch': 1.0}

In [88]:
predictions, labels, _ = trainer.predict(tokenized_datasets["train"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(
    predictions=true_predictions, 
    references=true_labels)
results

100%|██████████| 109/109 [02:45<00:00,  1.61s/it]

{'EUT': {'precision': 0.49191685912240185,
  'recall': 0.36613665663944994,
  'f1': 0.4198078344419808,
  'number': 2327},
 'UBJ': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 13618},
 'overall_precision': 0.49191685912240185,
 'overall_recall': 0.05343367826904986,
 'overall_f1': 0.09639644736097754,
 'overall_accuracy': 0.8085934754803435}